In [ ]:
from amplpy import AMPL, Environment

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix

from solution import sequential_savings_init

In [ ]:
pathEnv = "ABSOLUTE_PATH_AMPL_ENVIRONMENT"
pathModel = "ABSOLUTE_PATH_FLP_MODEL.MOD" #TAKE THE FLP_MODEL.MOD FILE THAT YOU FIND IN THIS FOLDER
pathData = "ABSOLUTE_PATH_DATA_FILE"

In [ ]:
ampl = AMPL(Environment(pathEnv))

ampl.option['ampl_include'] = 'models'
ampl.option['solver'] = 'cplex'
ampl.read(pathModel)
ampl.readData(pathData)

ampl.solve()

In [ ]:
n = ampl.getParameter('n').value()
minimart_range = ampl.getParameter('range').value()
Vc = ampl.getParameter('Vc').value()
Fc = ampl.getParameter('Fc').value()
capacity = ampl.getParameter('capacity').value()

Cx = ampl.getParameter('Cx').getValues().toList()
Cy = ampl.getParameter('Cy').getValues().toList()
Dc = ampl.getParameter('Dc').getValues().toList()
installed = ampl.getVariable('installed').getValues().toList()

total_cost = ampl.getObjective('obj').value()

cx_df = pd.DataFrame(Cx)
cy_df = pd.DataFrame(Cy)
dc_df = pd.DataFrame(Dc)
installed_df = pd.DataFrame(installed)

# final form of the dataframe
df = pd.DataFrame(columns=['Cx', 'Cy', 'Dc', 'installed'], dtype=np.float64)
df['Cx'] = cx_df[1]
df['Cy'] = cy_df[1]
df['Dc'] = dc_df[1]
df['installed'] = installed_df[1]


inst_df = df[ df['installed'] == 1 ]

inst_df


In [ ]:
coord_list = list()

for _, row in inst_df.iterrows():
    x_coord = row['Cx']
    y_coord = row['Cy']
    new_point = [ x_coord, y_coord ]
    coord_list.append(new_point)


demand_list = list()
demand_list.append(0)

for index in range(1, len(inst_df.index)):
    demand_list.append(1)


In [ ]:
dist = distance_matrix(coord_list, coord_list)

In [ ]:
plt.scatter(inst_df['Cx'], inst_df['Cy'])
plt.show()

In [ ]:
sol = sequential_savings_init(dist, demand_list, capacity)

In [ ]:
total_installation_cost = 0.0

for _, row in inst_df.iterrows():
    total_installation_cost += row['Dc']



total_refurbishment_cost = 0.0

for i in range(1, len(sol)):
    total_refurbishment_cost += dist[sol[i-1], sol[i]] * Vc
    
    if sol[i] == 0:
        total_refurbishment_cost += Fc

cost = total_installation_cost + total_refurbishment_cost

print(cost)
print(total_installation_cost)
print(total_refurbishment_cost)


In [ ]:
index_list = list()

for elem in inst_df.index:
    index_list.append(elem+1)

print(index_list)

routes_list = list()
for elem in sol:
    routes_list.append(index_list[elem])

print(sol)
print(routes_list)
